

*   This Notebook contains the code that was used to fine-tuned the models on the GLUE benchmark
*   Connect the Notebook to a GPU (e.g. T4 or A100) in order to run the code



# Environment settings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/MA2/Master Thesis/project')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12

In [ ]:
import numpy as np
import copy

import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

from transformers import Trainer, TrainingArguments
from transformers import (
    BertModel, BertForSequenceClassification, BertTokenizer, BertConfig,
    GPT2Model, GPT2ForSequenceClassification, GPT2Tokenizer, GPT2Config,
    RobertaModel, RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig,
)
from sklearn.metrics import accuracy_score
from nn_tools.nn_compression import Model, knapsack_greedy_compression, count_parameters
from nn_tools.fine_tuning import fine_tune, evaluate

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#The number of classes corresponding to each classification dataset from the GLUE benchmark
classification_dict = {
    "cola" : 2,
    "sst2" : 2,
    "mrpc" : 2,
    "qqp" : 2,
    "mnli" : 3,
    "qnli" : 2,
    "rte" : 2,
    "wnli" : 2,
}

# Fine tuning

In [ ]:
#Classification Model
name_ = ['bert', 'gpt2', 'roberta']
dataset_name_ = ['cola', 'mrpc', 'rte', 'qnli', 'wnli']
task = 'classification'

for name in name_ :
  for dataset_name in dataset_name_ :
    #Number of classes for the classification head
    n_class = classification_dict[dataset_name]

    #Load pretrained model without any specified weight path
    tuned_Model = Model(name, task, n_class=n_class)

    #Fine tune the model
    tuned_Model = fine_tune(tuned_Model, dataset_name, train_fraction=1, test_fraction=1, n_epochs=3, batch_size=16)

    #Save the model
    path = f"/content/drive/MyDrive/MA2/Master Thesis/project/models/{name}/{name}_{dataset_name}.pth"
    torch.save(tuned_Model.model, path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Step,Training Loss
500,0.540700
1000,0.487300
1500,0.468900
2000,0.452300
2500,0.505700
3000,0.511500
3500,0.510600
4000,0.610700
4500,0.696500
5000,0.696500


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/635 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Step,Training Loss
